In [ ]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import List, Optional
import os
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent


load_dotenv(find_dotenv())

class NumbersInput(BaseModel):
    """Input for arithmetic operations."""
    numbers: List[float] = Field(description="List of numbers to operate on")

@tool(args_schema=NumbersInput)
def calculate_sum(numbers_input: NumbersInput) -> float:
    """Calculate the sum of a list of numbers."""
    return sum(numbers_input.numbers)

@tool(args_schema=NumbersInput)  
def calculate_product(numbers_input: NumbersInput) -> float:
    """Calculate the product of a list of numbers."""
    result = 1
    for num in numbers_input.numbers:
        result *= num
    return result


api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_BASE_URL') or "https://api.openai.com/v1"
model_name = os.getenv('OPENAI_MODEL') or "gpt-4o-mini"

if api_key:
    llm = ChatOpenAI(
        api_key=api_key,
        base_url=base_url,
        model=model_name,
        temperature=0.1
    )
    print("Client initialized successfully!")
    print(f"Model: {model_name}")
    print(f"Base URL: {base_url}")
else:
    print("API key not found in environment variables")

tools = [calculate_sum, calculate_product]
system_prompt = "You are a helpful math assistant. Please solve problems provided by the user with only available tools"

agent = create_react_agent(
    model=llm,
    tools=tools,
    state_modifier=system_prompt,
    debug=True,
)

# Test the agent
inputs = {
    "messages": [
        {"role": "user", "content": "what is 2 times 35?"}
    ]
}

try:
    result = agent.invoke(inputs)
    print(f"Final result: {result['messages'][-1].content}")
except Exception as e:
    print(f"Error: {e}")